# Estudo e testes

In [2]:
# YOLOv8
import ultralytics
from ultralytics import YOLO

import cv2
import torch

In [4]:
print(cv2.__version__)

4.8.1


In [2]:
print('CUDA disponível' if torch.cuda.is_available() else 'CUDA indisponível')
print(f'\nUltralytics: {ultralytics.__version__}')
print(f'Torch: {torch.__version__}')

CUDA disponível

Ultralytics: 8.0.106
Torch: 2.1.1+cu121


In [3]:
class Point():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [4]:
# Carrega o vídeo
VIDEO = './dataset/road_video001.mp4'
OUTPUT = 'output_yolov8_tracker.mp4'

# Abrir o vídeo
cap = cv2.VideoCapture(VIDEO)

WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Inicializar o output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

START = Point(0, int(HEIGHT * 2/3))
END = Point(WIDTH, int(HEIGHT * 2/3))

# Cria o contador de objetos
veiculos = set()

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Carregando o modelo pré-treinado
model = YOLO('yolov8n.pt')

cuda


In [6]:
print(model.model.names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [7]:
# Selecionando apenas as classes relevantes pro problema
classes = [2, 7]

In [8]:
def track_and_count(output_path, tracker, linha):
    out = cv2.VideoWriter(output_path, fourcc, 25, (1280, 720))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model.track(frame, classes=classes, persist=True, tracker=tracker, device=0, verbose=False)
        
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xywh.cpu()
            tracks_ids = results[0].boxes.id.int().cpu().tolist()
            
            annotated_frame = results[0].plot()
                                    
            for box, track_id in zip(boxes, tracks_ids):
                x, y, w, h = box
                cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)
                
                if linha:
                    if START.x < x < END.x and abs(y - START.y) < 5:
                        veiculos.add(track_id)
                else:
                    veiculos.add(track_id)
        
        if linha:  
            cv2.line(annotated_frame, (START.x, START.y), (END.x, END.y), (0, 0, 255), 2)
        
        cv2.putText(annotated_frame, f'Veiculos: {len(veiculos)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        out.write(annotated_frame)

    cap.release()
    out.release()

    print(f'Veiculos: {len(veiculos)}')

# **Utilizando tracker ByteTrack**

#### Com linha

In [14]:
# ByteTrack com linha
track_and_count('outputs/output_bytetrack_linha.mp4', 'bytetrack.yaml', linha=True)

Veiculos: 68


#### Sem linha

In [9]:
# ByteTrack sem linha
track_and_count('outputs/output_bytetrack.mp4', 'bytetrack.yaml', linha=False)

Veiculos: 80


# **Utilizando tracker BoT-SORT (default)**

#### Com linha

In [9]:
# BoT-SORT com linha
track_and_count('outputs/output_botsort_linha.mp4', 'botsort.yaml', linha=True)

Veiculos: 68


#### Sem linha

In [9]:
# BoT-SORT sem linha
track_and_count('outputs/output_botsort.mp4', 'botsort.yaml', linha=False)

Veiculos: 80
